In [1]:
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:256'

dirs = ["../.."]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)

from swift.llm import (
    ModelType, get_vllm_engine, get_default_template_type,
    get_template, inference_vllm, VllmGenerationConfig, LoRARequest
)
from custom import CustomModelType, CustomTemplateType

lora_checkpoint = '/home/hanlv/workspace/code/research/infodemic/LLM/swift/examples/pytorch/llm/output/openchat_3.5/with_solar_info/brave/data1-split=8:2-ratio=1.0/dora-r=3/lr=8e-5-20240322-12:39:34/checkpoint-609'
lora_request = LoRARequest('default-lora', 1, lora_checkpoint)

model_type = CustomModelType.openchat_35
llm_engine = get_vllm_engine(
    model_type,
    tensor_parallel_size=1,
    seed=42,
    enable_lora=True,
    max_loras=1, 
    max_lora_rank=16,
    engine_kwargs={"max_num_seqs": 128},
)

template_type = get_default_template_type(model_type)
template = get_template(template_type, llm_engine.hf_tokenizer)


2024-03-22 20:33:23,675 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-03-22 20:33:23,678 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-03-22 20:33:23,714 - modelscope - INFO - Loading done! Current index file version is 1.13.1, with md5 ad87f4a589251d86333765b92ab59e41 and a total number of 972 components indexed
[INFO:swift] Loading the model using model_dir: /home/css/models/openchat-3.5-0106
[INFO:swift] Setting torch_dtype: torch.bfloat16
[INFO:swift] model_config: MistralConfig {
  "_name_or_path": "/home/css/models/openchat-3.5-0106",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e

INFO 03-22 20:33:24 llm_engine.py:87] Initializing an LLM engine with config: model='/home/css/models/openchat-3.5-0106', tokenizer='/home/css/models/openchat-3.5-0106', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 03-22 20:33:30 llm_engine.py:357] # GPU blocks: 3307, # CPU blocks: 2048
INFO 03-22 20:33:31 model_runner.py:684] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 03-22 20:33:31 model_runner.py:688] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 03-22 20:33:34 model_runner.py:756] Graph capturing finished in 3 secs.


In [2]:
generation_config = VllmGenerationConfig(
    max_new_tokens=512,
    temperature=0,
)
import jsonlines
data = []
with jsonlines.open(
f"../../my_data/with_solar_info/brave/train_test_split/8:2/test_data1.jsonl", 'r') as f:
    for item in f.iter():
        data.append(item)

request_list = [{'query': i["query"]} for i in data]

resp_list = inference_vllm(
    llm_engine, template, request_list, 
    generation_config=generation_config, 
    lora_request=lora_request,
    use_tqdm=True,
    # verbose=True, prompt_prefix="", output_prefix=""
)

# for request, resp in zip(request_list, resp_list):
#     print(f"query: {request['query']}")
#     print()
#     print(f"response: {resp['response']}")


  0%|          | 1/2439 [00:07<5:02:35,  7.45s/it]

KeyboardInterrupt: 